In [1]:
# 数据集的下载链接：https://storage.googleapis.com/kaggle-data-sets/10449/44567/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1584256867&Signature=sLZx3VJGj1sRQerPVTCo1PzIpxj75QpIqbK%2FUvDs7X9HBR2RXihITaxCp8gpgEZX7NvXB0aNhqbU20xRqf4rrVU9x2qmbF8RgG8BbwrUZi%2BXZ4iypwwzS8oOcNdXPZPm%2FNhNFE0MxzSOuETFBN1DGWBSWAjuJcakLqnZfIuGd5Np%2Fuolh0iUlLUaqcef4LFvoK6lhe8tdhKhaGRNncq5TwzhR%2FMyPwAS4t4r5IhydNjMPAQu6gUzA9OUzXicwkR73WILBEQ1DZu7BNyPWKi9oqGamMCe7KkojcljnJbX1vnRXmuT10iH%2FYzgvJmUA%2F7RKgMeio3Ch7cvptw8g7JmkA%3D%3D&response-content-disposition=attachment%3B+filename%3D10-monkey-species.zip

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf

from tensorflow import keras
import os, sys, time 


print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)



sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.1.0
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
train_dir = '../data/10-monkey-species/training/training'
valid_dir = '../data/10-monkey-species/validation/validation'
label_file = '../data/10-monkey-species/monkey_labels.txt'

print(os.path.exists(train_dir))
print(os.listdir(train_dir))
print(os.listdir(valid_dir))
# 定义参数
# 对于对10monkeys图片的分析，我们设定height和width设定图片读取出来之后，我们要缩放到什么大小
height = 128
width = 128
chaneels = 3
batch_size = 64
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, # 每个像素点除以 255，得到一个0-1之间的数，rgb的像素范围为0-255
    rotation_range = 40, # 将图片随机旋转到-40到40度之间
    width_shift_range = 0.2, # 设置位移当处理图片时我们将原数据和水平移动20%后的数据都传送进去，如果设置的是小于1的数，则表示比例，如果大于1则表示是像素值
    height_shift_range = 0.2, # 设置位移当处理图片时我们将原数据和垂直移动20%后的数据都传送进去，如果设置的是小于1的数，则表示比例，如果大于1则表示是像素值
    shear_range = 0.2, # 剪切程度
    zoom_range = 0.2, # 缩放成都
    horizontal_flip = True, # 是否随机作水平翻转
    fill_mode = 'nearest', # 像素填充，填充的规则是什么
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (height, width), # 设置图像的大小就是我们之前设置的128
    batch_size = batch_size, # 代表我们生成的图片多少为一组
    seed = 7, # 随机数，用来作随机
    shuffle = True, 
    class_mode = 'categorical' # 控制label的格式，两种 one-hot编码后的0/1，one-hot编码前的index，这里我们使用one-hot编码后的格式
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = 'categorical',
)
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

True
['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']
['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']
Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [ ]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
# x：64 为样本数，128，128为图像大小，3为通道数
# y：

In [3]:
# 创建各个层
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='selu', input_shape=[width, height, chaneels]),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='selu'),

    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='selu'),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='selu'),

    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='selu'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 128)       0

In [4]:
epochs = 10
model.sample_weight_mode = None
history = model.fit_generator(train_generator, steps_per_epoch=train_num//batch_size, epochs=epochs, validation_data=valid_generator, validation_steps=valid_num//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 17 steps, validate for 4 steps
Epoch 1/10
 1/17 [>.............................] - ETA: 8:24

ResourceExhaustedError:  OOM when allocating tensor with shape[524288,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MatMul_3 (defined at <ipython-input-4-8e2376647e2c>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_1233]

Function call stack:
distributed_function
